In [18]:
using Pkg
for p in ["Knet", "Plots", "IterTools"]
    if !haskey(Pkg.installed(),p)
        Pkg.add(p);
    end
end
using DelimitedFiles
using Knet: Adam, KnetArray
using IterTools: ncycle, takenth
using Base.Iterators: flatten

include("utils.jl")
include("layers.jl")
include("models.jl")

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at utils.jl:3
└ @ Core /home/ec2-user/dl/utils.jl:3


In [19]:
# Load data
adj, features, labels, idx_train, idx_val, idx_test = load()

(K64(2708,2708)[0.16666666666666666⋯], K32(1433,2708)[0.0⋯], Int32[0 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 1; 0 0 … 0 0], 1:140, 141:500, 501:2708)

In [20]:
# TODO: take user inputs
struct args
    epochs
    lr
    weight_decay
    hidden
    pdrop
end

arguments = args(200, 0.01, 5e-4, 16, 0.5)
# Model and optimizer
model = GCN(size(features,1),
            arguments.hidden,
            size(labels,2),
            arguments.pdrop)

MethodError: MethodError: no method matching KnetArray(::Float64)
Closest candidates are:
  KnetArray(!Matched::KnetArray{T,N}) where {T, N} at /home/ec2-user/.julia/packages/Knet/vxHRi/src/karray.jl:87
  KnetArray(!Matched::AbstractArray{T,N}) where {T, N} at /home/ec2-user/.julia/packages/Knet/vxHRi/src/karray.jl:93
  KnetArray(!Matched::AutoGrad.Value{A}) where A<:AbstractArray at /home/ec2-user/.julia/packages/Knet/vxHRi/src/karray.jl:388

In [10]:
output = model(KnetArray(features), convert(KnetArray{Float32,2}, KnetArray(adj)))

UndefVarError: UndefVarError: model not defined

In [ ]:
labels2 = mapslices(argmax, labels ,dims=2)

In [ ]:
output3 = convert(Array{Float64,2}, output)  

In [ ]:
output3[:,idx_train]

In [ ]:
nll(output3[:,idx_train], labels2[idx_train])

In [ ]:
model.layer1.w.value

In [ ]:
model.layer1.w.value

In [ ]:
using Knet: accuracy

In [ ]:
accuracy(output3[:,idx_train], labels2[idx_train])

In [ ]:
function train_with_results(model, features, epoch, lr)
    training = adam(model, ncycle(features,epoch))
    snapshot() = (deepcopy(model), model(features))
    snapshots = (snapshot() for x in takenth(progress(training),length(features)))
    lin = reshape(collect(flatten(snapshots)),(5,:))
    return lin
end